# Note on training:

Rather than data being the image and target being the correct value, the data will be three images and the target will be which image is more similar to the first image.

Create a custom dataset with PyTorch using the 3 images as the data.

In [1]:
# installs
!pip install torchviz
!pip install wandb

# Library imports
from __future__ import print_function
import numpy as np
import pandas as pd
import argparse
import wandb
import os
from collections import OrderedDict

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms, utils
from torch.optim.lr_scheduler import StepLR
from torchviz import make_dot

import matplotlib.pyplot as plt
from PIL import Image
import math
%matplotlib inline

from skimage import io, transform
from sklearn.metrics import roc_auc_score
# import umap
import seaborn as sns

from six.moves import urllib
opener = urllib.request.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0')]
urllib.request.install_opener(opener)

  Created wheel for torchviz: filename=torchviz-0.0.2-cp37-none-any.whl size=4152 sha256=557e5d942b010569b32d3057fae68dbf3dca1465460ece0911776e72d8916b88
  Stored in directory: /root/.cache/pip/wheels/01/26/58/026ffd533dbe8b3972eb423da9c7949beca68d1c98ed9e8624
Successfully built torchviz
     |████████████████████████████████| 1.8MB 8.2MB/s 
     |████████████████████████████████| 102kB 13.3MB/s 
     |████████████████████████████████| 174kB 50.6MB/s 
     |████████████████████████████████| 133kB 51.9MB/s 
     |████████████████████████████████| 71kB 11.4MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6489 sha256=42983d15b9564ab074898e85bd4ca22a7c9cd584cb2d3f35b3e2c5e41cafbcec
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=5e55a66c7ceb53c83308d8d013975ea2bca67df20ca36499df7877c426eb9594
  Stored

In [2]:
try:
    from google.colab import drive
    drive.mount('/content/gdrive')
    IN_COLAB = True
except:
    IN_COLAB = False

Mounted at /content/gdrive


# Meta-criteria

In [3]:
def metacriteria_attractive(A_attr: dict, B_attr: dict, C_attr: dict):
  A_attractive = A_attr['Attractive']
  B_attractive = B_attr['Attractive']
  C_attractive = C_attr['Attractive']

  if A_attractive == B_attractive:
    if A_attractive == C_attractive:
      return 0.5
    return 1
  if A_attractive == C_attractive:
    return 0
  return 0.5

def metacriteria_similar_hair(a, b, c):
  return 0

def metacriteria_similar_facial_hair(a, b, c):
  if(a == b and a != c):
    return 1
  if(a == b and a == c):
    return 0.5
  return 0

# Settings

In [4]:
# data settings:

n_test = 5000
# metacriteria = metacriteria_sameNumber
batch_size=64 #input batch size for training (default: 64)
batch_size_test=1000 


# model settings
epochs=40 #number of epochs to train (default: 14)
lr=3 #learning rate (default: 1.0)
gamma=0.7 #Learning rate step gamma (default: 0.7)
seed=42 #random seed (default: 42)
save_model=False #save the trained model (default: False)

# misc settings
no_cuda=False #disables CUDA training (default: True)
use_cuda = not no_cuda and torch.cuda.is_available()
torch.manual_seed(seed)
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {'num_workers': 0, 'pin_memory': True} if use_cuda else {}

print("Device:", device)

Device: cuda


### Data Directories

In [5]:
# Danny's Working Directory
dir = "/gdrive/MyDrive/Qualitative Research Assisted by AI"

# Danny's CelebA Paths
project_path = '/content/gdrive/MyDrive/Qualitative Research Assisted by AI/Summer 2021'
os.chdir(project_path)
os.listdir('Data/celeba')

encoderPath = '/content/gdrive/MyDrive/Qualitative Research Assisted by AI/Spring 2021/vae_encoder.pt'

#CelebA Dataset

### Get Dataset

In [6]:
# unzip data onto gcp instance storage for faster unzipping/loading
# this takes a minute, only do once per session
!mkdir ~/tmp
!cp Data/celeba/img_align_celeba.zip ~/tmp
!unzip ~/tmp/img_align_celeba.zip -d ~/tmp

# also copy over the attribute and partition data
!cp Data/celeba/list_eval_partition.csv ~/tmp
!cp Data/celeba/list_attr_celeba.csv ~/tmp

# set root directory
root_dir = '/root/tmp'

Streaming output truncated to the last 5000 lines.
 extracting: /root/tmp/img_align_celeba/197600.jpg  
 extracting: /root/tmp/img_align_celeba/197601.jpg  
 extracting: /root/tmp/img_align_celeba/197602.jpg  
 extracting: /root/tmp/img_align_celeba/197603.jpg  
 extracting: /root/tmp/img_align_celeba/197604.jpg  
 extracting: /root/tmp/img_align_celeba/197605.jpg  
 extracting: /root/tmp/img_align_celeba/197606.jpg  
 extracting: /root/tmp/img_align_celeba/197607.jpg  
 extracting: /root/tmp/img_align_celeba/197608.jpg  
 extracting: /root/tmp/img_align_celeba/197609.jpg  
 extracting: /root/tmp/img_align_celeba/197610.jpg  
 extracting: /root/tmp/img_align_celeba/197611.jpg  
 extracting: /root/tmp/img_align_celeba/197612.jpg  
 extracting: /root/tmp/img_align_celeba/197613.jpg  
 extracting: /root/tmp/img_align_celeba/197614.jpg  
 extracting: /root/tmp/img_align_celeba/197615.jpg  
 extracting: /root/tmp/img_align_celeba/197616.jpg  
 extracting: /root/tmp/img_align_celeba/197617.j

### Transforms

In [7]:
img_shape = (64, 64)

# transforms
tfms_train = transforms.Compose([
    transforms.Grayscale(),
    transforms.RandomHorizontalFlip(),
    transforms.Resize(img_shape),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

tfms_val = tfms_test = transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize(img_shape),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

###Dataset Class

In [8]:
class CelebA(Dataset):
  def __init__(self, root_dir: str, partition: str, transform=None):
    # the image data
    self.img_dir = f"{root_dir}/img_align_celeba"

    # the partition data
    if partition=='train':
      self.partition_ix = 0
    elif partition=='val':
      self.partition_ix = 1
    elif partition=='test':
      self.partition_ix = 2
    else:
      raise ValueError("partition must be one of 'train', 'val', or 'test'")

    df_partitions = pd.read_csv(f"{root_dir}/list_eval_partition.csv")
    self.df_partitions = df_partitions[df_partitions['partition']==self.partition_ix]

    # the attribute data
    df_attributes = pd.read_csv(f"{root_dir}/list_attr_celeba.csv")

    # filter the attributes
    self.df_attributes = self.df_partitions.merge(df_attributes, how='left', on='image_id').drop(columns=['partition'])

    # transforms
    self.transform = transform

  def __len__(self):
    return self.df_attributes.shape[0]

  def __getitem__(self, ix):
    # image
    filepath = f"{self.img_dir}/{self.df_attributes.image_id[ix]}"
    image = Image.open(filepath)
    if self.transform:
      image = self.transform(image)

    # attributes
    attributes =self.df_attributes.iloc[ix, 1:].to_dict()

    return {'image': image, 'attributes': attributes}


#Triplet dataset class
Where the inputs are:
*   List of triple indices to use for training
*   Original training set
*   Function for evaluating meta-criteria

And the output is:
*   ((A: image,B: image,C: image), target: bool)



### Defining the class

In [9]:
class ImageTripletDataset(Dataset):
    """Dataset of triplets of images"""

    def __init__(self, ImageDataset, num_triplets, criteria):
        """
        Args:
            ImageDataset (torch.utils.data.Dataset):  A pytorch dataset that serves individual images
            num_triplets (int): Number of triplets
        """
        self.imagedataset=ImageDataset
        self.num_triplets = num_triplets
        self.criteria = criteria

        # generate indices list
        self.indices = np.random.randint(0, len(self.imagedataset), (self.num_triplets, 3))
        # TODO: make sure there are no duplicates?

    def __len__(self):
        return self.num_triplets

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        Aix, Bix, Cix = self.indices[idx]

        A = self.imagedataset[Aix]
        B = self.imagedataset[Bix]
        C = self.imagedataset[Cix]

        A_img, A_attr = A['image'], A['attributes']
        B_img, B_attr = B['image'], B['attributes']
        C_img, C_attr = C['image'], C['attributes']

        sample = {'A': A_img, 'B': B_img, 'C':C_img, 
                  'target': self.criteria(A_attr,B_attr,C_attr),
                  'image_indices': (Aix, Bix, Cix), 'image_digits': (A_attr, B_attr, C_attr)}

        return sample

### Creating datasets/dataloaders for training and testing

# VAE Model

### Encoder 
VAE Encoder copied from a separate VAE Training Notebook

In [10]:
# add faces encoder
# encoder class copied from VAE notebook
class Encoder(nn.Module):
  def __init__(self, coarse_resolution = (4,4), hid_channels=(8, 12, 16)):
    super(Encoder, self).__init__()

    self.coarse_resolution = coarse_resolution
    
    # block 1:
    self.block1 = nn.Sequential(OrderedDict(
      conv = nn.Conv2d(in_channels=1, out_channels=hid_channels[0], kernel_size=5, stride=1, padding=2, bias=False),
      bn = nn.BatchNorm2d(hid_channels[0]),
      relu = nn.ReLU(),
      pool = nn.MaxPool2d(2)
    ))

    # block 2:
    self.block2 = nn.Sequential(OrderedDict(
      conv = nn.Conv2d(in_channels=hid_channels[0], out_channels=hid_channels[1], kernel_size=3, stride=1, padding=1, bias=False),
      bn = nn.BatchNorm2d(num_features=hid_channels[1]),
      relu = nn.ReLU(),
      pool = nn.MaxPool2d(2)
    ))

    # block 3:
    self.block3 = nn.Sequential(OrderedDict(
      conv = nn.Conv2d(in_channels=hid_channels[1], out_channels=hid_channels[2], kernel_size=3, stride=1, padding=1, bias=False),
      bn = nn.BatchNorm2d(num_features=hid_channels[2]),
      relu = nn.ReLU(),
      pool = nn.AdaptiveAvgPool2d(output_size=self.coarse_resolution),
      flatten = nn.Flatten()
    ))

  def forward(self, x):
    x = self.block1(x)
    x = self.block2(x)
    z = self.block3(x)

    return(z)


### Decoder 
VAE Decoder copied from a separate VAE Training Notebook

In [11]:
class Decoder(nn.Module):
  def __init__(self, coarse_resolution = (4,4), hid_channels=(16, 12, 8)):
    super(Decoder, self).__init__()

    self.coarse_resolution = coarse_resolution
    self.zdim = hid_channels[0]*self.coarse_resolution[0]*self.coarse_resolution[1]
    
    # block 1:
    self.block1 = nn.Sequential(OrderedDict(
      unflatten = nn.Unflatten(-1, (hid_channels[0], self.coarse_resolution[0], self.coarse_resolution[1])),
      conv = nn.Conv2d(in_channels=hid_channels[0], out_channels=hid_channels[1], kernel_size=3, stride=1, padding=1, bias=False),
      bn = nn.BatchNorm2d(hid_channels[1]),
      relu = nn.ReLU(),
      upsample = nn.UpsamplingBilinear2d(scale_factor=2)
    ))

    # block 2:
    self.block2 = nn.Sequential(OrderedDict(
      conv = nn.Conv2d(in_channels=hid_channels[1], out_channels=hid_channels[2], kernel_size=3, stride=1, padding=1, bias=False),
      bn = nn.BatchNorm2d(num_features=hid_channels[2]),
      relu = nn.ReLU(),
      upsample = nn.UpsamplingBilinear2d(scale_factor=2)
    ))

    # block 3:
    self.block3 = nn.Sequential(OrderedDict(
      conv = nn.Conv2d(in_channels=hid_channels[2], out_channels=hid_channels[2], kernel_size=3, stride=1, padding=1, bias=False),
      bn = nn.BatchNorm2d(num_features=hid_channels[2]),
      relu = nn.ReLU(),
      upsample = nn.UpsamplingBilinear2d(size=(28, 28))
    ))

    # touch-up 
    self.touchup = nn.Sequential(OrderedDict(
        conv = nn.Conv2d(in_channels=hid_channels[2], out_channels=1, kernel_size=3, stride=1, padding=1),
        sig = nn.Sigmoid()
    ))

  def forward(self, z):
    z = self.block1(z)
    z = self.block2(z)
    z = self.block3(z)
    x = self.touchup(z)

    return(x)

# Triplet model

### Triplet probability class

In [12]:
# From Van der Maaten: https://ieeexplore.ieee.org/abstract/document/6349720/?casa_token=_dWfybjO5O4AAAAA:T4Al2g3ZWaHcRwCzlp9QqRji1amJ-uCHwbEyAOHk3_AH9uIGjs4iBFaUf8XO-wqLpd7D2BH1eQ
# this has 0 free parameters
class TripletProbability(nn.Module):
  def __init__(self, alpha):
    super(TripletProbability, self).__init__()
    self.alpha=alpha

  def t_dist(self, d):
    return (1+d**2/self.alpha)**(-1*(self.alpha+1)/2)
  
  def forward(self, dAB, dAC):
    tAB = self.t_dist(dAB)
    tAC = self.t_dist(dAC)
    return tAB / (tAB + tAC)

### Pairwise-distance neural network module

In [13]:
class PairwiseDistance(nn.Module):
  def __init__(self, n_hid):
    super(PairwiseDistance, self).__init__()
    self.n_hid = n_hid

    self.f = nn.Sequential(
        nn.Linear(2*self.n_hid, self.n_hid),
        nn.ReLU(),
        nn.Linear(self.n_hid,1)
    )
  
  def forward(self, A, B):
    # A: [batch_size, n_hid]
    # B: [batch_size, n_hid]
    AB = torch.cat([A, B], dim=1)
    return self.f(AB).squeeze()

### TripletNet 
A neural network for modeling whether or not a triplet of images, (A, B, C), 
satisfies the proposition "A is more similar to B than C" according to a meta-
criteria. 

In [14]:
class TripletNet(nn.Module):
    def __init__(self, n_hid=10, alpha=1):
        super(TripletNet, self).__init__()

        self.n_hid=n_hid
        self.alpha=alpha

        # feature encoder
        # resuse for each input image
        self.encoder = Encoder()
        
        # distance computer: takes two samples and computes a distance
        # reuse this for pairs (A, B) and (A, C)
        # can make this more complex or more simple in future
        self.pairwise_distance = PairwiseDistance(n_hid=self.n_hid)

        # triplet probability computer defined in the class above
        self.triplet_probability = TripletProbability(self.alpha)
        
    def forward(self, A, B, C):
      # first compute all of the encodings
      A, B, C = [self.encoder(x) for x in (A,B,C)]
      
      # then get the pairwise distances
      dAB = self.pairwise_distance(A, B)
      dAC = self.pairwise_distance(A, C)
      
      # finally return the triplet probability
      return self.triplet_probability(dAB, dAC)

#Training and Testing

In [15]:
#@title Training and testing functions
def train(model, criteria, device, loader, optimizer):
  model.train()

  mean_batch_losses = []
  for batch_idx, batch_dict in enumerate(loader):
    A, B, C, target = [batch_dict[key].to(device) for key in ["A", "B", "C", "target"]]
    optimizer.zero_grad()
    output = model(A,B,C)
    loss = criteria(output.float(), target.float())
    loss.backward()
    optimizer.step()
    mean_batch_losses.append(loss.item())
        
  return np.mean(mean_batch_losses)

            
def test(model, criteria, device, loader):
    model.eval()

    mean_batch_losses = []
    outputs = []
    targets = []

    with torch.no_grad():
        for batch_idx, batch_dict in enumerate(loader):
          A, B, C, target = [batch_dict[key].to(device) for key in ["A", "B", "C", "target"]]
          output = model(A, B, C)
          loss = criteria(output.float(), target.float()) 

          # store results
          mean_batch_losses.append(loss.item())
          outputs.append(output)
          targets.append(target)

    outputs = torch.cat(outputs)
    targets = torch.cat(targets)


    
    return np.mean(mean_batch_losses), outputs, targets

# TRAINING LOOP

### Loop Helper Functions

In [16]:
def makeData(n_train, metacriteria):
  #training data
 # image_dataset_train = dataPath
  image_dataset_train = CelebA(root_dir, "train", transform = tfms_train)
  triplet_dataset_train = ImageTripletDataset(image_dataset_train, n_train, metacriteria)
  train_loader = torch.utils.data.DataLoader(
      triplet_dataset_train,
      batch_size=batch_size, shuffle=True, **kwargs)
    
  # testing data
  image_dataset_test = CelebA(root_dir, "test", transform = tfms_test)
  triplet_dataset_test = ImageTripletDataset(image_dataset_test, n_test, metacriteria)
  test_loader = torch.utils.data.DataLoader(
      triplet_dataset_test,
      batch_size=batch_size_test, shuffle=True, **kwargs)
    
  return train_loader,test_loader



def makeModel():
  model = TripletNet(n_hid=256).to(device)
  encoder_state_dict = torch.load(encoderPath)
  model.encoder.load_state_dict(encoder_state_dict)
  
  return model



def getLowestError(model, train_loader, test_loader):
  optimizer = optim.Adadelta(model.parameters(), lr=lr)
  scheduler = StepLR(optimizer, step_size=1, gamma=gamma)
  criteria = nn.BCELoss()

  best_test = 10**10
  best_train = 10**10
  best_auroc_filtered = 0.5
  best_auroc_all = 0.5
  best_accuracy_all = 0.5
  best_accuracy_filtered = 0.5
  best_epoch = 0
  for epoch in range(1, epochs + 1):
    train_loss = train(model, criteria, device, train_loader, optimizer)
    test_loss, outputs, targets = test(model, criteria, device, test_loader)
    scheduler.step()

    auroc_filtered = getAUROC_filtered(outputs, targets)
    auroc_all = getAUROC_all(outputs, targets)
    accuracy_filtered = getAccuracy_filtered(outputs, targets)
    accuracy_all = getAccuracy_all(outputs,targets)
   
    wandb.log({"Datasize": n_train, "Test_Loss": test_loss, "AUROC_filtered": auroc_filtered, 
               "Accuracy_filtered": accuracy_filtered, "Accuracy_all": accuracy_all, "Epoch": epoch})
    
    print("Train Loss: %0.3f. Test Loss: %0.3f. AUROC_Filtered: %0.3f. AUROC_All: %0.3f. Accuracy Filtered: %0.3f. Accuracy All: %0.3f. Epoch: %i" % (train_loss, test_loss, auroc_filtered, auroc_all, accuracy_filtered, accuracy_all, epoch))
    if test_loss<best_test:
      best_test = test_loss
      best_train = train_loss.item()
      best_auroc_filtered = auroc_filtered
      best_auroc_all = auroc_all
      best_accuracy_filtered = accuracy_filtered
      best_accuracy_all = accuracy_all
      best_epoch = epoch
    if epoch > best_epoch + 5:
      break

  #return best_test, best_auroc, best_epoch
  return best_test, best_epoch, best_accuracy_all, best_accuracy_filtered, best_auroc_filtered, best_auroc_all

def getAUROC_filtered(outputs, targets):
  ix_keep = targets!=0.5
  filteredOutputs = outputs[ix_keep]
  filteredTargets = targets[ix_keep]
  return roc_auc_score(filteredTargets.cpu().numpy(), filteredOutputs.cpu().numpy())

def getAUROC_all(outputs, targets):
  predictions = outputs.cpu().numpy()
  targets = targets.cpu().numpy()
  binned_predictions = []
  binned_targets = []

  bins = [0, 1/2, 1]
  bin_indices = np.digitize(predictions, bins)
  for index in bin_indices:
    if index == 1:
      binned_predictions.append(0)
    elif index == 2:
      binned_predictions.append(1)

  bin_indices = np.digitize(targets, bins)
  for target in targets:
    if target == 0:
      binned_targets.append(0)
    elif target == 0.5:
      binned_targets.append(1)
    elif target == 1:
      binned_targets.append(1)
  
  return roc_auc_score(binned_targets, binned_predictions)

def getAccuracy_filtered(outputs, targets):
  ix_keep = targets!=0.5
  filteredOutputs = outputs[ix_keep]
  filteredTargets = targets[ix_keep]
  filteredOutputs = filteredOutputs.cpu().numpy()
  filteredTargets = filteredTargets.cpu().numpy()

  correct = 0;
  results = np.equal(filteredOutputs, filteredTargets)
  for result in results:
    if result:
      correct += 1
  
  return (correct / filteredTargets.size)


def getAccuracy_all(predictions, targets):
  predictions = predictions.cpu().numpy()
  targets = targets.cpu().numpy()
  new_predictions = []
  correct = 0

  bins = [0, 1/3, 2/3, 1]
  bin_indices = np.digitize(predictions, bins)
  for index in bin_indices:
    if index == 1:
      new_predictions.append(0)
    elif index == 2:
      new_predictions.append(0.5)
    elif index == 3:
      new_predictions.append(1)
  
  results = np.equal(new_predictions,targets)
  for result in results:
    if result:
      correct += 1
  
  return (correct / predictions.size)

### The Loop

In [17]:
metacriteria_list = [metacriteria_attractive]                    
trainingSize = np.logspace(2, 3.69897000434, 15).astype(np.int)


for metacriteria in metacriteria_list:

  run = wandb.init(project='qualitative-analysis', entity='witw', config = {
        "metacriteria": metacriteria.__name__, "pretrained": True}, reinit = True)
  wandb.run.name = "CELEBA3-pretrained" + metacriteria.__name__ 

  for n_train in trainingSize:
    
    print("TRAINING SIZE: %i", n_train)
    train_loader,test_loader = makeData(n_train, metacriteria)
    
    # make model
    model = makeModel()
    
    #test model
    best_test, best_epoch, best_accuracy_all, best_accuracy_filtered, best_auroc_filtered, best_auroc_all = getLowestError(model,train_loader,test_loader)
    wandb.log({'Datasize': n_train,
               'Test_Loss': best_test, 
               'AUROC_filtered': best_auroc_filtered, 
               'Accuracy_filtered': best_accuracy_filtered, 
               'Accuracy_all': best_accuracy_all, 
               'Epoch': best_epoch})
    
    print("Datasize: %i", n_train)    
    print("Best Test Loss: %0.3f. Best AUROC Filtered: %0.3f. Best AUROC All: %0.3f. Best Accuracy Filtered: %0.3f. Best Accuracy All: %0.3f. Best Epoch: %i\n" % (best_test, best_auroc_filtered, best_auroc_all, best_accuracy_filtered, best_accuracy_all, best_epoch))


    if save_model:
        torch.save(model.state_dict(), "mnist_flat.pt")

  run.finish()




<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


TRAINING SIZE: %i 100
Train Loss: 0.693. Test Loss: 0.693. AUROC_Filtered: 0.505. AUROC_All: 0.501. Accuracy Filtered: 0.000. Accuracy All: 0.500. Epoch: 1
Train Loss: 0.693. Test Loss: 0.693. AUROC_Filtered: 0.504. AUROC_All: 0.499. Accuracy Filtered: 0.000. Accuracy All: 0.500. Epoch: 2
Train Loss: 0.693. Test Loss: 0.693. AUROC_Filtered: 0.498. AUROC_All: 0.503. Accuracy Filtered: 0.000. Accuracy All: 0.500. Epoch: 3
Train Loss: 0.693. Test Loss: 0.693. AUROC_Filtered: 0.496. AUROC_All: 0.506. Accuracy Filtered: 0.000. Accuracy All: 0.500. Epoch: 4
Train Loss: 0.693. Test Loss: 0.693. AUROC_Filtered: 0.495. AUROC_All: 0.505. Accuracy Filtered: 0.000. Accuracy All: 0.500. Epoch: 5
Train Loss: 0.693. Test Loss: 0.693. AUROC_Filtered: 0.494. AUROC_All: 0.502. Accuracy Filtered: 0.000. Accuracy All: 0.500. Epoch: 6
Train Loss: 0.693. Test Loss: 0.693. AUROC_Filtered: 0.494. AUROC_All: 0.503. Accuracy Filtered: 0.000. Accuracy All: 0.500. Epoch: 7
Datasize: %i 100
Best Test Loss: 0.693. 

Datasize,5000
Test_Loss,0.6931
AUROC_filtered,0.49813
Accuracy_filtered,0.0
Accuracy_all,0.4972
Epoch,3
_runtime,3418
_timestamp,1621578010
_step,131


Datasize,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▃▃▃▃▃▄▄▄▅▅▅▆▆▆███
Test_Loss,▁▁▁▂▂▁▂▂▂▇▁▂▁▄▆▃█▁▃▄▁▅▁▃▃▂▅▁▁▄▅▁▄▃▇▃▁▂▄▁
AUROC_filtered,▆▄▆▇▆▅▅▅▆▅▆▅▅▃▃▄▄▇▅▆▇▆▇▇█▇▁▅▂▅▆▄▆▇▇▆█▄▆▅
Accuracy_filtered,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Accuracy_all,▄▄▄▃▃▁▁▁▆▆▄▄▄▄▄▃▃▄▄▄▄▄▄▁▁▁▃▃███▄▄▄▆▆▆▄▄▄
Epoch,▂▃▁▃▅▁▄▆▃▆▁▄▁▃▆▃▄▁▄▆▃▆▂▃▆▃▃▁▂▅█▁▄▇▁▄▂▂▅▃
_runtime,▁▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇██
_timestamp,▁▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇██
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███


In [ ]:
df

,Datasize,Test Loss,AUROC Filtered,Accuracy Filtered,Accuracy All,Epoch
0,100.0,0.692884,0.515545,0.0,0.0116,7.0
1,132.0,0.693082,0.560333,0.0,0.0108,20.0
2,174.0,0.693159,0.511655,0.0,0.0100,1.0
3,231.0,0.693108,0.518308,0.0,0.0106,4.0
4,305.0,0.693174,0.469963,0.0,0.0100,1.0
5,404.0,0.693128,0.515908,0.0,0.0086,3.0
6,534.0,0.693034,0.520580,0.0,0.0106,3.0
7,707.0,0.693158,0.491591,0.0,0.0088,2.0
8,935.0,0.692584,0.615256,0.0,0.0142,6.0
9,1236.0,0.692410,0.530975,0.0,0.0094,3.0


# Visualize the embeddings

In [ ]:
def get_encodings(model, device, loader):
    model.eval()

    outputs = []
    targets = []
    with torch.no_grad():
        for batch_idx, (img, target) in enumerate(loader):
          output = model(img.to(device))
          outputs.append(output)
          targets.append(target)

    outputs = torch.cat(outputs)
    targets = torch.cat(targets)

    
    return outputs, targets

In [ ]:
encoder = model.encoder
image_loader_test = torch.utils.data.DataLoader(
    image_dataset_test,
    batch_size=batch_size_test, shuffle=True, **kwargs)

outputs, targets = get_encodings(encoder, device, image_loader_test)

NameError: ignored

In [ ]:
test_embeddings = outputs.cpu().numpy()
targets = targets.cpu().numpy()

In [ ]:
reducer = umap.UMAP()
embedding = reducer.fit_transform(test_embeddings)
embedding.shape

In [ ]:
plt.figure(figsize=(15,15))
sns.scatterplot(x=embedding[:,0], y=embedding[:,1], hue=targets.astype(str), hue_order=[str(i) for i in range(10)])
plt.show()

In [ ]:
pd.concat([pd.DataFrame(embedding), pd.DataFrame(targets)], axis = 1).to_csv(dataPath + "least_common_multiple_UMAP.csv")

In [ ]:
def experiment_criteria(metacriteria, epochs=3, dataset_train=False, dataset_test=False, n_train=50000, n_test=5000, batch_size=64,batch_size_test=1000, UMAP=True):
  #settings
  lr=1
  gamma=0.7
  seed=42
  no_cuda=False
  use_cuda = not no_cuda and torch.cuda.is_available()
  torch.manual_seed(seed)
  device = torch.device("cuda" if use_cuda else "cpu")
  kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

  if not dataset_train and dataset_test:
    #download training data if not included
    dataset_train = datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ]))
    # download testing data if not included
    dataset_test = datasets.MNIST('../data', train=False, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ]))
  #train data
  triplet_dataset_train = ImageTripletDataset(dataset_train, n_train, metacriteria)
  train_loader = torch.utils.data.DataLoader(
    triplet_dataset_train,
    batch_size=batch_size, shuffle=True, **kwargs)
  
  #test data
  triplet_dataset_test = ImageTripletDataset(dataset_test, n_test, metacriteria)
  test_loader = torch.utils.data.DataLoader(
    triplet_dataset_test,
    batch_size=batch_size_test, shuffle=True, **kwargs)
  
  model = TripletNet().to(device)

  optimizer = optim.Adadelta(model.parameters(), lr=lr)
  scheduler = StepLR(optimizer, step_size=1, gamma=gamma)
  criteria = nn.BCELoss()

  for epoch in range(1, epochs + 1):
    train_loss = train(model, criteria, device, train_loader, optimizer)
    test_loss, outputs, targets = test(model, criteria, device, test_loader)
    scheduler.step()

    roc_auc = roc_auc_score(targets.cpu().numpy(), outputs.cpu().numpy())

    print("Train loss: %0.3f. Test loss: %0.3f. AUROC: %0.3f" % (train_loss, test_loss, roc_auc))

  if UMAP:
    encoder = model.encoder
    image_loader_test = torch.utils.data.DataLoader(
      dataset_test,
      batch_size=batch_size_test, shuffle=True, **kwargs)

    outputs, targets = get_encodings(encoder, device, image_loader_test)
    
    test_embeddings = outputs.cpu().numpy()
    targets = targets.cpu().numpy() 

    reducer = umap.UMAP()
    embedding = reducer.fit_transform(test_embeddings)
    embedding.shape

    plt.figure(figsize=(15,15))
    sns.scatterplot(x=embedding[:,0], y=embedding[:,1], hue=targets.astype(str), hue_order=[str(i) for i in range(10)])
    plt.show()

In [ ]:
experiment_criteria(metacriteria_least_common_multiple, epochs=6,dataset_train=image_dataset_train, dataset_test=image_dataset_test)